In [1]:
import nltk
import librosa
import torch
import soundfile as sf
from transformers import Wav2Vec2ForCTC,Wav2Vec2Tokenizer
import evaluate
nltk.download('punkt')

2023-12-03 12:14:26.902973: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-03 12:14:27.063597: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 12:14:27.063628: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 12:14:27.064551: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 12:14:27.131327: I tensorflow/core/platform/cpu_feature_g

True

In [2]:
def load_model():
    """
    Returns the Wav2Vec2 tokienizer and the model from pretrained tokinezer model
    """
    tokienizer=Wav2Vec2Tokenizer.from_pretrained('facebook/wav2vec2-base-960h')
    model=Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
    return tokienizer, model

def Correction(input_text:str):
    """
    Returns the input text with correction

    Args:
        input_text (str): Original text generated by the model
    """
    sentence=nltk.sent_tokenize(input_text)
    return (" ".join([s.replace(s[0],s[0].capitalize(),1) for s in sentence]))


In [3]:
def transcribe(tokenizer,model,input_file):
    """
    Returns the transcribe text of the audio

    Args:
        tokiener (_type_): _description_
        model (_type_): _description_
        input_file (_type_): _description_
    """

    speech, sample_rate=sf.read(input_file)

    if len(speech.shape)>1:
        speech=speech[:,0]+speech[:,1]
    
    if sample_rate!=16000:
        speech=librosa.resample(speech,orig_sr=sample_rate,target_sr=16000)
    
    
    input_values = tokenizer(speech, return_tensors="pt").input_values
    logits=model(input_values).logits

    predicted_ids=torch.argmax(logits,dim=-1)

    transcription=tokenizer.decode(predicted_ids[0])

    transcription=Correction(transcription.lower())
    return transcription


In [8]:
wav_input ="common_voice_en_38024625.mp3" #"84-121123-0000.flac"
tokenizer, model = load_model()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/home/deon-xavier-pereira/Desktop/Project/NNDL/nndl_env/lib/python3.10/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:733: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. ini

In [9]:
text = transcribe(tokenizer,model,wav_input)
print(text)

The town ob line is served by the kohama county school district


In [6]:
print(wav_input)

84-121123-0000.flac


In [7]:
# Load evaluators
wer = evaluate.load('wer')
cer = evaluate.load('cer')

# Evaluate
wer_value = round(wer.compute(predictions=predictions_all, references=references_all), 4)
cer_value = round(cer.compute(predictions=predictions_all, references=references_all), 4)

# Print results
print('Final:')
print(f'WER: {wer_value} | CER: {cer_value}')

NameError: name 'predictions_all' is not defined